# Axisym PTST with unit ligament area

In [89]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC , TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from bmcs_matmod.slide.vslide_34_TN_axisym import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic

# FE axisymmetric discretization - Applying lateral compression/tension

## Area of the section = 1, load = 1

Expected $\sigma_{x} =  \frac{Load}{Area} =  \frac{Load}{1} = Load$ 
Expected $u_x = \frac{\sigma_{x}}{E} = \frac{Load}{10000} = Load 1e^{-4}$ 

In [153]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGridAxiSym(coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
xd_upper = XDomainFEGridAxiSym(coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          integ_factor=2 * np.pi,
                          fets=FETS2D4Q())
m1 = MATS3DElastic(E=10000, nu=0.3)
m2 = MATS3DElastic(E=10000, nu=0.3)

xd12 = XDomainFEInterface(
    integ_factor = 2 * np.pi * R_in,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [154]:
material_params =  dict(
     E_T=10000, gamma_T=500, K_T=0, S_T=0.5, c_T=6, bartau=3, 
     E_N=10000, S_N=0.05, c_N = 3, m = 0., f_t=2, f_c=60, f_c0 = 40, eta=0.)
bond_m = Slide34(**material_params)

In [155]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ]
)

In [156]:
lower_fixed_1 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
lower_fixed_0 = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0], var='u', dims=[0], value=0)

upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, -1, :, -1],
         var='u', dims=[1], value=-0.1)
compression_dofs = upper_compression_slice.dofs
compression = 100 / len(compression_dofs)
upper_compression_force_first = [BCDof(var='f', dof=dof, value = compression) 
             for dof in compression_dofs ]

bc1 =  upper_compression_force_first + [lower_fixed_0, lower_fixed_1]

In [157]:
m.bc=bc1

m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
s.tstep.fe_domain.serialized_subdomains

In [158]:
#bond_m.interact()

In [159]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



In [160]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [161]:
time = m.hist.t
_, (ax1, ax2) = plt.subplots(1,2)
ax1.plot(time, sig_pi_N[:,0,0].flatten(), label='sigma')
ax1.plot(time, sig_pi_N[:,0,1].flatten(), label='sigma')
ax1.set_xlabel('time')
ax1.set_ylabel('normal stress')
ax1.legend()

ax2.plot(time, sig_pi_Tx[:,0,0].flatten(), color='red', label='tau')
ax2.set_xlabel('time')
ax2.set_ylabel('tangential stress')
ax2.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [152]:
max(sig_pi_N[:,0,0])

52.53727159897743

In [111]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

# FE 2D discretization - Applying lateral compression/tension

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC , TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface
from ibvpy.fets import FETS2D4Q
from ibvpy.fets.fets1D5 import FETS1D52ULRH
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from bmcs_matmod.slide.vslide_34_TN_axisym import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic

In [190]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGrid(integ_factor = P,
                         coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          fets=FETS2D4Q())
xd_upper = XDomainFEGrid(integ_factor = P,
                         coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          fets=FETS2D4Q())
m1 = MATS2DElastic(E=37000, nu=0.18)
m2 = MATS2DElastic(E=37000, nu=0.18)

xd12 = XDomainFEInterface(
    integ_factor = P,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [191]:
material_params =  dict(
     E_T=1000, gamma_T=500, K_T=0, S_T=0.5, c_T=1, bartau=3, 
     E_N=1000, S_N=0.00005, c_N = 3, m = 0., f_t=2, f_c=60, f_c0 = 40, eta=0.)
bond_m = Slide34(**material_params)

In [192]:
# lower_fixed_1 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
# lower_fixed_0 = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0], var='u', dims=[0], value=0)
    
# upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, -1, :, -1],
#          var='u', dims=[1], value=0)
# compression_dofs = upper_compression_slice.dofs
# compression = -100 / len(compression_dofs)
# upper_compression_force_first = [BCDof(var='f', dof=dof, value = compression) 
#              for dof in compression_dofs ]
# bc1 =  upper_compression_force_first + [lower_fixed_0, lower_fixed_1]

In [193]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ]
)

In [194]:
lower_fixed_1 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
lower_fixed_0 = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0], var='u', dims=[0], value=0)

upper_compression_slice = BCSlice(slice=xd_upper.mesh[:, -1, :, -1],
         var='u', dims=[1], value=-0.1)
compression_dofs = upper_compression_slice.dofs
compression = 10 / len(compression_dofs)
upper_compression_force_first = [BCDof(var='f', dof=dof, value = compression) 
             for dof in compression_dofs ]
bc1 =  upper_compression_force_first + [lower_fixed_0, lower_fixed_1]

In [195]:
upper_compression_slice.dofs

array([11, 15])

In [196]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
s.tstep.fe_domain.serialized_subdomains

In [197]:
#bond_m.interact()

In [198]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



In [199]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [200]:
xd12.o_Epia

array([[[[ 2,  3],
         [ 6,  7]],

        [[ 8,  9],
         [12, 13]]]])

In [201]:
U_last = m.hist.U_t[-1]
U_last[xd12.o_Epia]

array([[[[-3.97576090e-20,  4.30148495e-05],
         [-4.86486486e-05,  4.30148495e-05]],

        [[-4.04734842e-18,  1.00430148e-02],
         [-4.86486486e-05,  1.00430148e-02]]]])

In [202]:
xd12.map_U_to_field(U_last)

array([[[-4.00759081e-18,  1.00000000e-02],
        [-3.78115508e-18,  1.00000000e-02]]])

In [203]:
time = m.hist.t
f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4)
ax1.plot(time, sig_pi_N[:,0,0].flatten(), label='sigma')
ax1.plot(time, sig_pi_N[:,0,1].flatten(), label='sigma')
ax1.set_xlabel('time')
ax1.set_ylabel('normal stress')
ax1.legend()

ax2.plot(time, sig_pi_Tx[:,0,0].flatten(), color='red', label='tau')
ax2.set_xlabel('time')
ax2.set_ylabel('tangential stress')
ax2.legend()

ax3.plot(time, omega_Nx[:,0,0].flatten(), color='red', label='normal damage')
ax3.set_xlabel('time')
ax3.set_ylabel('damage')
ax3.legend()

ax4.plot(time, u_pi_N[:,0,0].flatten(), color='red', label='opening')
ax4.set_xlabel('time')
ax4.set_ylabel('opening')
ax4.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
sig_pi_N.shape

(11, 1, 2)

In [256]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

# FE 2D discretization - Applying sliding dc

In [162]:
n_y_e = 1
n_inner_x_e = 1
n_outer_x_e = 1
L_x = 1.0
P = 1.0
R_in = P / (2 * np.pi) 
R_out = 2 * R_in
xd_lower = XDomainFEGrid(integ_factor = P,
                         coord_min=(0, 0),
                          coord_max=(L_x, R_in),
                          shape=(n_y_e, n_inner_x_e),
                          fets=FETS2D4Q())
xd_upper = XDomainFEGrid(integ_factor = P,
                         coord_min=(0, R_in),
                          coord_max=(L_x, R_out),
                          shape=(n_y_e, n_outer_x_e),
                          fets=FETS2D4Q())
m1 = MATS2DElastic(E=37000, nu=0.18)
m2 = MATS2DElastic(E=37000, nu=0.18)

xd12 = XDomainFEInterface(
    integ_factor = P,
    I=xd_lower.mesh.I[:, -1],
    J=xd_upper.mesh.I[:, 0],
    fets=FETS1D52ULRH()
)

In [163]:
material_params =  dict(
     E_T=1000, gamma_T=500, K_T=0, S_T=0.5, c_T=1, bartau=3, 
     E_N=1000, S_N=0.00005, c_N = 3, m = 0., f_t=2, f_c=60, f_c0 = 40, eta=0.)
bond_m = Slide34(**material_params)

In [164]:
m = TStepBC(
    domains=[(xd_lower, m1),
             (xd_upper, m2),
             (xd12, bond_m),
             ]
)

In [176]:
lower_fixed_1 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
lower_fixed_0 = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0], var='u', dims=[0], value=0)
upper_fixed_1 = BCSlice(slice=xd_upper.mesh[:, -1, :, -1], var='u', dims=[1], value=0)

upper_slide_slice = BCSlice(slice=xd_upper.mesh[:, -1, :, -1],
         var='u', dims=[0], value=1)

bc1 =  [lower_fixed_0, lower_fixed_1,upper_fixed_1, upper_slide_slice]

In [177]:
upper_compression_slice.dofs

array([ 9, 13])

In [178]:
# lower_fixed_1 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
# lower_fixed_0 = BCSlice(slice=xd_lower.mesh[0, 0, 0, 0], var='u', dims=[0], value=0)
    
# upper_compression_slice = BCSlice(slice=xd_lower.mesh[:, -1, :, -1],
#          var='u', dims=[1], value=0)
# compression_dofs = upper_compression_slice.dofs
# compression = -100 / len(compression_dofs)
# upper_compression_force_first = [BCDof(var='f', dof=dof, value = compression) 
#              for dof in compression_dofs ]
# bc1 =  upper_compression_force_first + [lower_fixed_0, lower_fixed_1]

In [179]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
s.tstep.fe_domain.serialized_subdomains

In [180]:
#bond_m.interact()

In [181]:
xd12.hidden = True
s.reset()
s.run()

t:	 0.00(4), 
	 0.10(8), 
	 0.20(8), 
	 0.30(7), 
	 0.40(7), 
	 0.50(7), 
	 0.60(7), 
	 0.70(7), 
	 0.80(7), 
	 0.90(7), 
	 1.00(7), 



In [182]:
states_t = [states_t[2] for states_t in m.hist.state_vars]
var_names = states_t[0].keys()
EpsSig_t = {
    var_name: np.array([ state_dict[var_name] for state_dict in states_t ]) 
    for var_name in var_names 
}
u_pi_N = EpsSig_t['w_pi']
u_pi_Tx = EpsSig_t['s_pi_x']
sig_pi_N = EpsSig_t['sig_pi']
sig_pi_Tx = EpsSig_t['tau_pi_x']
sig_pi_Ty = EpsSig_t['tau_pi_y']
omega_Nx = EpsSig_t['omega_N']
omega_Tx = EpsSig_t['omega_T']
x_m = xd12.x_Eia[:,:,0].flatten()

In [183]:
xd12.o_Epia

array([[[[ 2,  3],
         [ 6,  7]],

        [[ 8,  9],
         [12, 13]]]])

In [184]:
U_last = m.hist.U_t[-1]
U_last[xd12.o_Epia]

array([[[[ 6.06222552e-05, -1.10837616e-06],
         [ 2.32834729e-04, -1.07183022e-05]],

        [[ 9.99974402e-01,  4.78141545e-06],
         [ 9.99974469e-01, -4.82850990e-06]]]])

In [185]:
xd12.map_U_to_field(U_last)

array([[[9.99913779e-01, 5.88979160e-06],
        [9.99741634e-01, 5.88979234e-06]]])

In [189]:
time = m.hist.t
f, (ax1, ax2, ax3, ax4) = plt.subplots(1,4)
ax1.plot(time, sig_pi_N[:,0,0].flatten(), label='sigma')
ax1.plot(time, sig_pi_N[:,0,1].flatten(), label='sigma')
ax1.set_xlabel('time')
ax1.set_ylabel('normal stress')
ax1.legend()

ax2.plot(time, sig_pi_Tx[:,0,0].flatten(), color='red', label='tau')
ax2.set_xlabel('time')
ax2.set_ylabel('tangential stress')
ax2.legend()

ax3.plot(time, omega_Tx[:,0,0].flatten(), color='red', label='tangential damage')
ax3.set_xlabel('time')
ax3.set_ylabel('damage')
ax3.legend()

ax4.plot(time, u_pi_N[:,0,0].flatten(), color='red', label='opening')
ax4.set_xlabel('time')
ax4.set_ylabel('opening')
ax4.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
sig_pi_N.shape

(11, 1, 2)

In [256]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()